# Preliminary analysis
This notebook opens the file named 'responses_scrubbed_tagged.csv', which is created by running the 'free_text_tagging.ipynb' notebook, and does an automatic preliminary analysis of some statistics in the responses therein.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex
df = pd.read_csv('responses_complete.csv', index_col=None, parse_dates=['date'])

#don't truncate results
pd.set_option('display.max_rows', None)

## Number of respondents from each neighborhood
Gives us a sense of who responded.

In [2]:
# absolute values
df['neighborhood'].value_counts()

Harmon                 166
Upper Village          118
Old Post Road North     77
Mount Airy              67
Sunset Park             48
Albany Post Road        31
Half Moon Bay           21
Quaker Ridge            11
Cortlandt               10
Ossining                 1
Name: neighborhood, dtype: int64

In [3]:
# as percent
percent = df['neighborhood'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
percent

Harmon                 30.2%
Upper Village          21.5%
Old Post Road North    14.0%
Mount Airy             12.2%
Sunset Park             8.7%
Albany Post Road        5.6%
Half Moon Bay           3.8%
Quaker Ridge            2.0%
Cortlandt               1.8%
Ossining                0.2%
Name: neighborhood, dtype: object

## Number of respondents from each demographic
Gives us a sense of who responded.

In [4]:
# absolute values
df['demographic'].value_counts()

Adult with school kid       232
Adult without school kid    174
Senior                      140
College kid                   5
Middle School kid             3
High School kid               3
Name: demographic, dtype: int64

In [5]:
# as percent
df['demographic'].value_counts(normalize=True).mul(100).round(0).astype(str) + '%'

Adult with school kid       42.0%
Adult without school kid    31.0%
Senior                      25.0%
College kid                  1.0%
Middle School kid            1.0%
High School kid              1.0%
Name: demographic, dtype: object

## Percent respondents of each demographic from each neighborhood
Each column adds up to 100%.

In [6]:
df2 = df.groupby(['demographic', 'neighborhood']).size().unstack(fill_value=0).apply(lambda x: 100 * x / x.sum()) #.astype(int)
df2.round(0).astype(int).astype(str) + '%' # round to int, the slap on percent sign

neighborhood,Albany Post Road,Cortlandt,Half Moon Bay,Harmon,Mount Airy,Old Post Road North,Ossining,Quaker Ridge,Sunset Park,Upper Village
demographic,,,,,,,,,,
Adult with school kid,19%,40%,14%,40%,55%,45%,0%,55%,48%,43%
Adult without school kid,29%,40%,43%,27%,24%,33%,100%,27%,38%,33%
College kid,0%,0%,0%,2%,0%,1%,0%,0%,2%,0%
High School kid,0%,0%,0%,1%,1%,0%,0%,0%,0%,0%
Middle School kid,0%,0%,0%,1%,0%,1%,0%,0%,0%,1%
Senior,52%,20%,43%,30%,19%,20%,0%,18%,12%,23%


## Analyze questions with only one answer option allowed


In [7]:
# a function that calculates how many responses were given in a specific column
def get_num_respondents(field, df):
    # get the number of responses in this field
    num_respondents = df[pd.notnull(df[field])].shape[0]
    return num_respondents

def get_breakdown(field, breakdown_field, percent=True):
    # get the number of respondents to this field
    num_respondents = get_num_respondents(field, df)

    # get the overall stats of the values in the field of interest
    df2 = df[field].value_counts()
    if percent:
        df2 = df2.apply(lambda x: int(100 * x / num_respondents))
    
    # label the column nicely 
    df2 = df2.to_frame()
    df2.columns = ['overall'] #name this 'overall', since it is not broken down

    # get the various values given in the field we want to break down the target field by
    breakdowns = df[breakdown_field].value_counts().items()

    # now break down the field by each of the values in the breakdown field
    for breakdown, value in breakdowns:        
        # get the number of responses by respondents matching this breakdown category
        df3 = df[df[breakdown_field] == breakdown][field].value_counts()
        
        # convert to percentage, if desired
        if percent:
            # get the number of respondents to this field
            num_respondents_column = get_num_respondents(field, df[df[breakdown_field] == breakdown][field].to_frame())
            
            df3 = df3.apply(lambda x: int(100 * x / num_respondents_column))

        # label the column nicely before we merge it into the other dataframe
        df3 = df3.to_frame() # convert to dataframe
        df3.columns = [breakdown]
        

        # join these two dataframes together
        df2 = df2.join(df3, how='outer')

    # add percent signs if calculating percentages
    if percent:
        df2 = df2.round(0).fillna(0).astype(int).astype(str) + '%'
        
    # sort by index names to be consistent
    #df2 = df2.sort_values(by='overall', ascending=False)
    df2 = df2.sort_index()

    return df2

# fields of interest
fields = [ 'child_bus_freq', 'child_walk_freq', 'child_driven_freq', 'child_drive_freq', 'child_bike_freq', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk', 'self_jog_frequency', 'commutes', 'child_self_school', 'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq', 'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes', 'child_self_has_children', 'commuter_type', 'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq', 'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq', 'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq', 'drivers_are_safe', 'bicyclists_are_safe', 'contact_interest' ]
# not included: ['child_drive_reason', 'child_no_walk_reason', 'child_no_bike_reason', 'commuter_distance', 'no_walk_reason', 'no_bike_reason', 'drive_reason' ]

# add in the tagging fields, if desired
fields.extend([ 'problem_tags_129', 'problem_tags_aggressive driving', 'problem_tags_benedict', 'problem_tags_bushes on sidewalk', 'problem_tags_cet', 'problem_tags_cleveland', 'problem_tags_cpa', 'problem_tags_crossing guard', 'problem_tags_croton commons', 'problem_tags_cyclists breaking rules', 'problem_tags_driver awareness', 'problem_tags_dummy light', 'problem_tags_five corners', 'problem_tags_gottwald circle', 'problem_tags_grand', 'problem_tags_hmb trail', 'problem_tags_maple', 'problem_tags_missing crosswalks', 'problem_tags_missing sidewalks', 'problem_tags_morningside', 'problem_tags_mt airy', 'problem_tags_municipal place', 'problem_tags_n riverside', 'problem_tags_narrow roads', 'problem_tags_olcott', 'problem_tags_old post n', 'problem_tags_old post s', 'problem_tags_on-street parking', 'problem_tags_poor lighting', 'problem_tags_road surface', 'problem_tags_s riverside', 'problem_tags_shoprite', 'problem_tags_sidewalk condition', 'problem_tags_speeding', 'problem_tags_truesdale', 'problem_tags_van wyck', 'schools_CET', 'schools_CHHS', 'schools_Homeschooled', 'schools_PVC', 'suggestion_tags_better lighting', 'suggestion_tags_bike lanes', 'suggestion_tags_bike racks', 'suggestion_tags_dedicated pedestrian walk signals', 'suggestion_tags_educate cyclists', 'suggestion_tags_educate pedestrians', 'suggestion_tags_enforce crosswalks', 'suggestion_tags_enforce speed', 'suggestion_tags_enforce trimming of bushes', 'suggestion_tags_improve route 9 access', 'suggestion_tags_maintain crosswalks', 'suggestion_tags_maintain sidewalks', 'suggestion_tags_more crosswalks', 'suggestion_tags_more sidewalks', 'suggestion_tags_more signs', 'suggestion_tags_more stop signs', 'suggestion_tags_more traffic lights', 'suggestion_tags_no bike lanes', 'suggestion_tags_outlaw cycling', 'suggestion_tags_parking on one side only', 'suggestion_tags_reduce on-street parking', 'suggestion_tags_reduce speed', 'suggestion_tags_sidewalk on mt airy s', 'suggestion_tags_speed bumps', 'suggestion_tags_speed cameras', 'suggestion_tags_speed indicator signs', 'suggestion_tags_traffic calming' ])

# iterate through every field of interest
for field in fields:
    
    # different fields by which we will break down each column in the data set... add as many as we think are interesting
    breakdowns = {
        'demographic': get_breakdown(field, 'demographic', percent=True),
        'neighborhood': get_breakdown(field, 'neighborhood', percent=True),
        'commuter_type': get_breakdown(field, 'commuter_type', percent=True),
        'bike_freq': get_breakdown(field, 'bike_freq', percent=True),
        'walk_freq': get_breakdown(field, 'walk_freq', percent=True)
    }
    
    # print out results overall in general on average
    display(Markdown('## *{}* (each column adds up to 100%):'.format(field, get_num_respondents(field, df)).title()))

    # break down the results by each of the breakdown fields
    for breakdown_field, results in breakdowns.items():        
        display(Markdown('### {} broken down by {}:'.format(field, breakdown_field)))
        display(results)


## *Child_Bus_Freq* (Each Column Adds Up To 100%):

### child_bus_freq by demographic:

,overall,Adult with school kid,Adult without school kid,Senior,College kid,Middle School kid,High School kid
"Every day, or almost every day",49%,49%,0%,0%,0%,0%,0%
Once every few days,3%,3%,0%,0%,0%,0%,0%
Once in a while,6%,6%,0%,0%,0%,0%,0%
Rarely or never,40%,40%,0%,0%,0%,0%,0%


### child_bus_freq by neighborhood:

,overall,Harmon,Upper Village,Old Post Road North,Mount Airy,Sunset Park,Albany Post Road,Half Moon Bay,Quaker Ridge,Cortlandt,Ossining
"Every day, or almost every day",49%,40%,47%,52%,64%,47%,50%,33%,33%,75%,0%
Once every few days,3%,7%,0%,5%,2%,4%,0%,0%,0%,0%,0%
Once in a while,6%,10%,6%,5%,8%,0%,0%,0%,0%,0%,0%
Rarely or never,40%,41%,45%,35%,24%,47%,50%,66%,66%,25%,0%


...

,overall,I take the train at Croton Harmon,I drive the entire way,I walk the entire way (weather permitting),I bicycle the entire way (weather permitting),I take a bus the entire way,I take the train at another station
0,99%,98%,100%,100%,100%,100%,50%
1,0%,1%,0%,0%,0%,0%,50%


### suggestion_tags_traffic calming by bike_freq:

,overall,Rarely or never,Once in a while,Once every few days,"Every day, or almost every day"
0,99%,99%,99%,100%,93%
1,0%,0%,0%,0%,6%


### suggestion_tags_traffic calming by walk_freq:

,overall,"Every day, or almost every day",Once every few days,Once in a while,Rarely or never
0,99%,98%,99%,99%,100%
1,0%,1%,0%,0%,0%


## Who rides on the sidewalk?

### Overall

In [8]:
# do people generally bike on the sidewalk?
df['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bikes_on_sidewalk
No - I don't bicycle,48%
No - I only bicycle on the road,29%
Yes,23%


### People who never bike

In [9]:
# do people who never bike claim to ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Rarely or never']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bikes_on_sidewalk
No - I don't bicycle,81%
No - I only bicycle on the road,12%
Yes,7%


### People who bike at least occasionally

In [10]:
# do people who ride a bike occasionally or more ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days', 'Once in a while']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bikes_on_sidewalk
No - I don't bicycle,1%
No - I only bicycle on the road,53%
Yes,46%


### People who bike a lot

In [11]:
# do people who bike a lot ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bikes_on_sidewalk
No - I only bicycle on the road,60%
Yes,40%


## Who thinks drivers are safe?

### Overall

In [12]:
# do people generally think drivers are safe?
df['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,9%
2.0,29%
3.0,48%
4.0,14%


### People who frequently walk

In [13]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,10%
2.0,32%
3.0,47%
4.0,10%


### People who infrequently walk

In [14]:
# do people who rarely walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,6%
2.0,24%
3.0,48%
4.0,22%


### People who rarely walk

In [15]:
# do people who never walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,9%
2.0,19%
3.0,43%
4.0,30%


### People who frequently bike 

In [16]:
# do people who bike a lot think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,12%
2.0,31%
3.0,49%
4.0,8%


### People who infrequently bike

In [17]:
# do people who never bike think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,8%
2.0,29%
3.0,47%
4.0,16%


### People who rarely bike

In [18]:
# do people who never bike think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Rarely or never']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,drivers_are_safe
1.0,6%
2.0,29%
3.0,47%
4.0,18%


## Who thinks bicyclists are safe?

### Overall

In [19]:
# do people generally think bicyclists are safe?
df['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bicyclists_are_safe
1.0,13%
2.0,23%
3.0,49%
4.0,16%


### People who frequently bike

In [20]:
# do people who bike a lot think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bicyclists_are_safe
1.0,4%
2.0,14%
3.0,56%
4.0,26%


### People who infrequently bike

In [21]:
# do people who never bike think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bicyclists_are_safe
1.0,15%
2.0,25%
3.0,46%
4.0,13%


### People who rarely bike

In [22]:
# do people who never bike think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Rarely or never']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,16
2.0,29
3.0,44
4.0,11


### People who frequently walk

In [23]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bicyclists_are_safe
1.0,11%
2.0,23%
3.0,50%
4.0,16%


### People who infrequently walk

In [24]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bicyclists_are_safe
1.0,17%
2.0,23%
3.0,46%
4.0,14%


### People who rarely walk

In [25]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index().astype(str) + '%'


,bicyclists_are_safe
1.0,26%
2.0,19%
3.0,45%
4.0,11%
